# 1. Set Up

In [17]:
# !spack find -lvp tfel
# !echo "$(spack find -p tfel | awk '/\/mofem_install\// {print $NF "/lib"}')"
# %env LD_LIBRARY_PATH=/mofem_install/spack/opt/spack/linux-ubuntu20.04-x86_64/gcc-9.4.0/tfel-4.0.0-mvfpqw7u4c23su7hj7g4leuwmykrjmcx/lib

In [ ]:
import math
import os
import re
import sys
import time
import json
from pathlib import Path
import subprocess
import zipfile
import pydantic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyvista as pv
import gmsh

sys.path.append(f"{os.environ['HOME']}/lateral_capacity_pipeline/src")

import setup
import core
import custom_models as cm
import utils as ut
import plotting
import calculations as calc
import postprocess
    
import matplotlib
matplotlib.rc('figure', figsize=(7, 7))

# 2. Simulation Parameters

In [ ]:
#in MPa
def initialize_parameters(custom_mesh_filepath: str = None, soil_model: cm.PropertyTypeEnum = cm.PropertyTypeEnum.le, **kwargs) -> cm.AttrDict:
    params = cm.AttrDict()
    setattr(params, "custom_mesh_filepath", custom_mesh_filepath)
    setattr(params, "soil_model", soil_model)
    for key,value in kwargs.items():
        setattr(params, key, value)
    params.interface = False
    params.pile_manager = cm.PileManager(x=0, y=0, z=0, dx=0, dy=0, R=1, r=0.975,
                                         stickup_height = 10,
                                         embedded_depth = -10.5,
                                preferred_model= cm.PropertyTypeEnum.le,
                                props = {
                                    cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=200000, poisson_ratio=0.3),
                                },
                                interface = params.interface,
                                )

    soil_layer_1 = cm.SoilLayer(
        depth = -2,
        preferred_model= params.soil_model,
        props = {
            cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=96, poisson_ratio=0.49),
            cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(youngs_modulus=96, poisson_ratio=0.49, HardeningSlope = 1, YieldStress = 0.3),
            cm.PropertyTypeEnum.vM_Implicit_mfront: cm.VonMisesProperties(youngs_modulus=96, poisson_ratio=0.49, HardeningSlope = 1, YieldStress = 0.3),
            cm.PropertyTypeEnum.DP: cm.DruckerPragerProperties(youngs_modulus=96, poisson_ratio=0.35, phi=np.radians(15), c=1, v=np.radians(15)),
            cm.PropertyTypeEnum.DP_HYPER: cm.DPHYPERProps(youngs_modulus=96, poisson_ratio=0.35, phi=np.radians(15), c=1, v=np.radians(15), proximity = 0.5),
            cm.PropertyTypeEnum.MCC: cm.CamClayProperties(),
            }, 
            )
    soil_layer_2 = cm.SoilLayer(
        depth = -1.4,
        preferred_model= params.soil_model,
        props = {
            cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=182.1, poisson_ratio=0.49),
            cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(youngs_modulus=182.1, poisson_ratio=0.49, HardeningSlope = 1, YieldStress = 0.3),
            cm.PropertyTypeEnum.vM_Implicit_mfront: cm.VonMisesProperties(youngs_modulus=182.1, poisson_ratio=0.49, HardeningSlope = 1, YieldStress = 0.3),
            cm.PropertyTypeEnum.DP: cm.DruckerPragerProperties(youngs_modulus=182.1, poisson_ratio=0.35, phi=np.radians(15), c=1, v=np.radians(15)),
            cm.PropertyTypeEnum.DP_HYPER: cm.DPHYPERProps(youngs_modulus=182.1, poisson_ratio=0.35, phi=np.radians(15), c=1, v=np.radians(15), proximity = 0.5),
            cm.PropertyTypeEnum.MCC: cm.CamClayProperties(),
            }, 
            )
    soil_layer_3 = cm.SoilLayer(
        depth = -7.1,
        preferred_model= params.soil_model,
        props = {
            cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=351.3, poisson_ratio=0.49),
            cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(youngs_modulus=351.3, poisson_ratio=0.49, HardeningSlope = 1, YieldStress = 0.3),
            cm.PropertyTypeEnum.vM_Implicit_mfront: cm.VonMisesProperties(youngs_modulus=182.1, poisson_ratio=0.49, HardeningSlope = 1, YieldStress = 0.3),
            cm.PropertyTypeEnum.DP: cm.DruckerPragerProperties(youngs_modulus=351.3, poisson_ratio=0.35, phi=np.radians(15), c=1, v=np.radians(15)),
            cm.PropertyTypeEnum.DP_HYPER: cm.DPHYPERProps(youngs_modulus=351.3, poisson_ratio=0.35, phi=np.radians(15), c=1, v=np.radians(15), proximity = 0.5),
            cm.PropertyTypeEnum.MCC: cm.CamClayProperties(),
                 }, 
            )
    soil_layer_4 = cm.SoilLayer(
        depth = -29.5,
        preferred_model= params.soil_model,
        props = {
            cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=668.4, poisson_ratio=0.49),
            cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(youngs_modulus=668.4, poisson_ratio=0.49, HardeningSlope = 1, YieldStress = 0.3),
            cm.PropertyTypeEnum.vM_Implicit_mfront: cm.VonMisesProperties(youngs_modulus=668.4, poisson_ratio=0.49, HardeningSlope = 1, YieldStress = 0.3),
            cm.PropertyTypeEnum.DP: cm.DruckerPragerProperties(youngs_modulus=668.4, poisson_ratio=0.35, phi=np.radians(15), c=1, v=np.radians(15)),
            cm.PropertyTypeEnum.DP_HYPER: cm.DPHYPERProps(youngs_modulus=668.4, poisson_ratio=0.35, phi=np.radians(15), c=1, v=np.radians(15), proximity = 0.5),
            cm.PropertyTypeEnum.MCC: cm.CamClayProperties(),
            }, 
            )
    
    # params.interface_manager = cm.InterfaceManager(
    #     preferred_model = cm.PropertyTypeEnum.vM,
    #     props = {
    #         cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=96, poisson_ratio=0.49),
    #         cm.PropertyTypeEnum.vM: cm.VonMisesProperties(youngs_modulus=96, poisson_ratio=0.49,HardeningSlope = 5000, YieldStress = 20),
    #         cm.PropertyTypeEnum.DP: cm.DruckerPragerProperties(youngs_modulus=668.4, poisson_ratio=0.49, phi=np.radians(0.05), c=0, v=np.radians(0.05)),
    #         },
    #         )
    
    #=======================================================
    #=======================================================
    #=======================================================
    #if the soil domain is a cuboid
    params.box_manager = cm.BoxManager(x=-80, y=-80, z=0, dx=160, dy=80,
        layers=[
        soil_layer_1,
        soil_layer_2,
        soil_layer_3,
        soil_layer_4,   
        ],
        far_field_size=5,
        near_field_dist=5,
        near_field_size=0.05,
    )
    #if the soil domain is a half cylinder
    params.cylinder_manager = cm.CylinderManager(
        x=0, 
        y=0, 
        z=0, 
        dx=0, 
        dy=0, 
        r=80,
        pile_R = params.pile_manager.R,
        pile_r = params.pile_manager.r,
        layers=[
        soil_layer_1,
        soil_layer_2,
        soil_layer_3,
        soil_layer_4,   
        ],
        mesh_radial_progression = 1.3,
        mesh_radial_divisions = 26,
    )
    #=======================================================
    #=======================================================
    #=======================================================
    
    def custom_generate_physical_groups_mfront(params) -> list[cm.PhysicalGroup]:
        physical_group_list = [
            cm.PhysicalGroup(
                meshnet_id=2,
                name='MFRONT_MAT_2',
                preferred_model=cm.PropertyTypeEnum.vM_Default_mfront,
                props={cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(
                    youngs_modulus=31.3394184,
                    poisson_ratio=0.49,
                    HardeningSlope=0.0,
                    YieldStress=0.12437000000000001
                )}
            ),
            cm.PhysicalGroup(
                meshnet_id=3,
                name='MFRONT_MAT_3',
                preferred_model=cm.PropertyTypeEnum.vM_Default_mfront,
                props={cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(
                    youngs_modulus=94.3043148,
                    poisson_ratio=0.49,
                    HardeningSlope=0.0,
                    YieldStress=0.213515
                )}
            ),
            cm.PhysicalGroup(
                meshnet_id=4,
                name='MFRONT_MAT_4',
                preferred_model=cm.PropertyTypeEnum.vM_Default_mfront,
                props={cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(
                    youngs_modulus=157.87311480000002,
                    poisson_ratio=0.49,
                    HardeningSlope=0.0,
                    YieldStress=0.26967
                )}
            ),
            cm.PhysicalGroup(
                meshnet_id=5,
                name='MFRONT_MAT_5',
                preferred_model=cm.PropertyTypeEnum.vM_Default_mfront,
                props={cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(
                    youngs_modulus=222.04581840000003,
                    poisson_ratio=0.49,
                    HardeningSlope=0.0,
                    YieldStress=0.21247599999999997
                )}
            ),
            cm.PhysicalGroup(
                meshnet_id=6,
                name='MFRONT_MAT_6',
                preferred_model=cm.PropertyTypeEnum.vM_Default_mfront,
                props={cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(
                    youngs_modulus=286.9495632,
                    poisson_ratio=0.49,
                    HardeningSlope=0.0,
                    YieldStress=0.18651399999999999
                )}
            ),
            cm.PhysicalGroup(
                meshnet_id=7,
                name='MFRONT_MAT_7',
                preferred_model=cm.PropertyTypeEnum.vM_Default_mfront,
                props={cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(
                    youngs_modulus=352.5207804,
                    poisson_ratio=0.49,
                    HardeningSlope=0.0,
                    YieldStress=0.19436399999999998
                )}
            ),
            cm.PhysicalGroup(
                meshnet_id=8,
                name='MFRONT_MAT_8',
                preferred_model=cm.PropertyTypeEnum.vM_Default_mfront,
                props={cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(
                    youngs_modulus=418.69590120000004,
                    poisson_ratio=0.49,
                    HardeningSlope=0.0,
                    YieldStress=0.20269199999999998
                )}
            ),
            cm.PhysicalGroup(
                meshnet_id=9,
                name='MFRONT_MAT_9',
                preferred_model=cm.PropertyTypeEnum.vM_Default_mfront,
                props={cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(
                    youngs_modulus=485.5384944,
                    poisson_ratio=0.49,
                    HardeningSlope=0.0,
                    YieldStress=0.21110399999999999
                )}
            ),
            cm.PhysicalGroup(
                meshnet_id=10,
                name='MFRONT_MAT_10',
                preferred_model=cm.PropertyTypeEnum.vM_Default_mfront,
                props={cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(
                    youngs_modulus=553.04856,
                    poisson_ratio=0.49,
                    HardeningSlope=0.0,
                    YieldStress=0.26432
                )}
            ),
            cm.PhysicalGroup(
                meshnet_id=11,
                name='MFRONT_MAT_11',
                preferred_model=cm.PropertyTypeEnum.vM_Default_mfront,
                props={cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(
                    youngs_modulus=621.2578824,
                    poisson_ratio=0.49,
                    HardeningSlope=0.0,
                    YieldStress=0.27891279999999996
                )}
            ),
            cm.PhysicalGroup(
                meshnet_id=12,
                name='MFRONT_MAT_12',
                preferred_model=cm.PropertyTypeEnum.vM_Default_mfront,
                props={cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(
                    youngs_modulus=690.1028928000001,
                    poisson_ratio=0.49,
                    HardeningSlope=0.0,
                    YieldStress=0.29364159999999995
                )}
            ),
            cm.PhysicalGroup(
                meshnet_id=13,
                name='MFRONT_MAT_13',
                preferred_model=cm.PropertyTypeEnum.vM_Default_mfront,
                props={cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(
                    youngs_modulus=759.64716,
                    poisson_ratio=0.49,
                    HardeningSlope=0.0,
                    YieldStress=0.30851999999999996
                )}
            ),
            cm.PhysicalGroup(
                meshnet_id=14,
                name='MFRONT_MAT_14',
                preferred_model=cm.PropertyTypeEnum.vM_Default_mfront,
                props={cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(
                    youngs_modulus=962.5587696,
                    poisson_ratio=0.49,
                    HardeningSlope=0.0,
                    YieldStress=0.32896559999999997
                )}
            ),
            cm.PhysicalGroup(
                meshnet_id=15,
                name='MFRONT_MAT_15',
                preferred_model=cm.PropertyTypeEnum.vM_Default_mfront,
                props={cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(
                    youngs_modulus=1301.8254552,
                    poisson_ratio=0.49,
                    HardeningSlope=0.0,
                    YieldStress=0.3652572
                )}
            ),
            cm.PhysicalGroup(
                meshnet_id=16,
                name='MFRONT_MAT_16',
                preferred_model=cm.PropertyTypeEnum.vM_Default_mfront,
                props={cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(
                    youngs_modulus=1647.8940024,
                    poisson_ratio=0.49,
                    HardeningSlope=0.0,
                    YieldStress=0.4022764
                )}
            ),
            cm.PhysicalGroup(
                meshnet_id=17,
                name='MFRONT_MAT_17',
                preferred_model=cm.PropertyTypeEnum.vM_Default_mfront,
                props={cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(
                    youngs_modulus=2000.8915488,
                    poisson_ratio=0.49,
                    HardeningSlope=0.0,
                    YieldStress=0.4400368
                )}
            ),
            cm.PhysicalGroup(
                meshnet_id=18,
                name='MFRONT_MAT_18',
                preferred_model=cm.PropertyTypeEnum.vM_Default_mfront,
                props={cm.PropertyTypeEnum.vM_Default_mfront: cm.VonMisesProperties(
                    youngs_modulus=2360.945232,
                    poisson_ratio=0.49,
                    HardeningSlope=0.0,
                    YieldStress=0.478552
                )}
            ),
            cm.PhysicalGroup(
            meshnet_id=1,
            name='MFRONT_MAT_1',
            preferred_model=cm.PropertyTypeEnum.le,
            props={cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=200000, poisson_ratio=0.3)},
            ),
        ]


        return physical_group_list
    
    def custom_generate_physical_groups_mofem(params) -> list[cm.PhysicalGroup]:
        physical_group_list = []
        return physical_group_list
    
    
    if getattr(params, "custom_mesh_filepath", False):
        try:
            if params.use_mfront:
                params.custom_generate_physical_groups = custom_generate_physical_groups_mfront
            else:
                params.custom_generate_physical_groups = custom_generate_physical_groups_mofem
        except NameError as e:
            raise NameError("Must provided the custom callback 'custom_generate_physical_groups' to inject attributes to the main commmand.")
   
    params.nproc = 32 # number of processors/cores used
    params.dim = 3
    
    params.wk_dir = Path(f"{params.home}/mfront_example_test")
    if params.use_mfront:
        params.read_med_exe = f"{params.home}/um_view/bin/read_med"
        params.h5m_to_vtk_converter = f"{params.home}/um_view/bin/convert.py"
        params.partition_exe = f"{params.home}/um_view/bin/mofem_part"
        params.exe = f"{params.home}/um_view/tutorials/adv-1/contact_3d"
    else:
        params.read_med_exe = f"{params.home}/um_view_release/bin/read_med"
        params.h5m_to_vtk_converter = f"{params.home}/um_view_release/bin/convert.py"
        params.partition_exe = f"{params.home}/um_view_release/bin/mofem_part"
        params.exe = f"{params.home}/um_view_release/adolc_plasticity/adolc_plasticity_3d"
    params.paraview_path = "{params.home}/ParaView-5.13.1-MPI-Linux-Python3.10-x86_64/bin/pvpython"
    params.ffmpeg_path = f'{params.home}/ffmpeg-7.0.2-amd64-static/ffmpeg'
    params.preset_dir = f'{params.home}/mfront_example_test/src/presets'
    
    return params

In [ ]:
case_name = "pile_CM3"

if case_name.startswith("pile_CM2"):
    outer_radius = 0.381
    inner_radius = 0.371
    stickup_height = 10
    embeded_depth = -2.3
elif case_name.startswith("pile_CM9"):
    outer_radius = 0.381
    inner_radius = 0.368
    stickup_height = 10
    embeded_depth = -4.0
elif case_name.startswith("pile_CM3"):
    outer_radius = 0.381
    inner_radius = 0.356
    stickup_height = 10
    embeded_depth = -7.6

elif case_name.startswith("pile_CL2"):
    outer_radius = 1
    inner_radius = 0.975
    stickup_height = 10
    embeded_depth = -10.5
else:
    raise ValueError("Case name not recognized")
points_of_interest = [
    cm.Point(x=outer_radius,y=0,z=stickup_height),
    cm.Point(x=outer_radius,y=0,z=0),
    cm.Point(x=-outer_radius,y=0,z=0),
]
line = cm.Line(
    pt1 = cm.Point(x=-(outer_radius+inner_radius)/2,y=0,z=0),
    pt2 = cm.Point(x=-(outer_radius+inner_radius)/2,y=0,z=embeded_depth),
)

params = initialize_parameters(
    order = 2, #order of approximation functions
    case_name = case_name,
    # custom_mesh_filepath="/mofem_install/jupyter/thomas/mfront_example_test/user_data/mesh/pile/pile_CL2_vM_adolc_10.cub.cub", 
    base = "hex",
    # soil_model=cm.PropertyTypeEnum.vM_Default_mfront,
    # soil_model=cm.PropertyTypeEnum.vM_Implicit_mfront,
    # soil_model=cm.PropertyTypeEnum.DP,
    soil_model=cm.PropertyTypeEnum.le_adolc,
    # soil_model=cm.PropertyTypeEnum.vM_adolc,
    # soil_model=cm.PropertyTypeEnum.Hm_adolc,
    use_mfront = False,
    final_time = 1, # [s]
    time_step = 1 / 1, # [s]
    b_bar = False,
    # time_history = cm.TimeHistory(history = {
    #     0: 0, 
    #     0.95: 1,
    #     1: 0,
    #     }
    # ),
    # body_time_history = cm.TimeHistory(history = {
    #     0: 0, 
    #     0.01: 1, 
    #     1: 1,
    #     }
    # ),
    home = os.environ['HOME'],
    options_file = f"{os.environ['HOME']}/lateral_capacity_pipeline/param_file.petsc",
    
    save_gauss = 0,
    convert_gauss = 0,
    FEA_completed=True,
    days_since_epoch=153,
    sim_otd = 101,
    
    # prescribed_force = cm.ForceBoundaryCondition(fx=0,fy=0,fz=0),
    prescribed_disp = cm.SurfaceBoundaryCondition(disp_ux=1, disp_uy=None ,disp_uz=None),
    
    points_of_interest = points_of_interest,
    prescribed_BC_name = "FIX_X_1",
    
) # 2. Simulation Parameters    
params = setup.setup(params) # 3. Log paths and meta
if params.FEA_completed:
    # core.generate_mesh(params) # 4. Generate the mesh
    # core.export_to_vtk(params)
    # postprocess.zip_vtks(params)
    # postprocess.extract_log(params)
    # postprocess.snes_csv(params)
    # postprocess.extract(params)
    postprocess.line_to_csv(params, line)
    # postprocess.calculate(params)
    # postprocess.plot(params)
    pass
else:
    core.generate_mesh(params) # 4. Generate the mesh
    core.mofem_compute(params) # 5. Running the analysis and export to .vtk file format
    core.export_to_vtk(params)
    postprocess.zip_vtks(params)
    postprocess.extract_log(params)
    postprocess.snes_csv(params)
    postprocess.extract(params)
    postprocess.line_to_csv(params, line)
    postprocess.calculate(params)
    postprocess.plot(params)
    pass

╭──────────────────────────────────────────────────────────────────────────╮
│  PULLING A SELECTED LINE OVER DEPTH AT THE FINAL TIMESTEP WITH pvpython  │
╰──────────────────────────────────────────────────────────────────────────╯



Done, taken Wall Time: 0.35 seconds, CPU Time: 0.35 seconds
